In [36]:
import numpy as np
import scipy.special
import scipy.ndimage
import matplotlib.pyplot as plt
import scipy.misc
import glob
import copy
import random 
import datetime
import _pickle as pickle
import sys
sys.path.append('C:\\Users\\Mishan\\appdata\\local\\programs\\python\\python37-32\\lib\\site-packages')
from tqdm import tqdm
from PIL import Image, ImageDraw
DATE = datetime.datetime.today().strftime("%Y-%m-%d-%H-%M") #для названий файлов

In [37]:
#NETWORK ADRESS
ADRESS1 = "C:/Users/Mishan/Documents/NNL/"
ADRESS2 = "C:/Users/Acer/Documents/NNL/"
ADRESS3 = "C:/Users/Мишан/Documents/NNL/"
ADRESS = ADRESS1

In [38]:
def bw_photo(numphoto, factor = 9):
    #SPECIAL FUNCTION FOR TURNING PHOTO INTO B&W
    #takes number of photo
    mode = 5 #ЧБ преобразование. 
    photopath = ADRESS + "PHOTOS/photo"+str(numphoto)+".jpg"
    bwpath = ADRESS + "/BWPHOTOS/photo"+str(numphoto)+".png"
    image = Image.open(photopath) #Открываем изображение. 
    draw = ImageDraw.Draw(image) #Создаем инструмент для рисования. 
    width = image.size[0] #Определяем ширину. 
    height = image.size[1] #Определяем высоту. 	
    pix = image.load() #Выгружаем значения пикселей
    #factor = 9
    for i in tqdm(range(width)):
        for j in range(height):
            a = pix[i, j][0]
            b = pix[i, j][1]
            c = pix[i, j][2]
            S = a + b + c
            if (S > (((255 + factor) // 2) * 3)):
                a, b, c = 255, 255, 255
            else:
                a, b, c = 0, 0, 0
            draw.point((i, j), (a, b, c))
    image.save(bwpath, "PNG")
    del draw

In [39]:
def resize_function():
    #SPECIAL CELL FOR RESIZING RAW IMAGES TO TESTPICS
    rawpath = glob.glob(ADRESS + "/RAWPICS/raw*")
    for i in range(len(rawpath)):
        img = Image.open(rawpath[i])
        feature = "0000"+str(i+1)
        newpath =  ADRESS + "TESTPICS/testpic"+feature[len(feature)-5:]+".png"
        width = 28
        height = 28
        resized_img = img.resize((width, height), Image.ANTIALIAS)
        resized_img.save(newpath)

In [40]:
def draw_number(number,database = False):
    if database:
        data_file = open(ADRESS + "/CSV/mnist_train.csv",'r')
        data_list = data_file.readlines()
        data_file.close()
        all_values = data_list[number].split(',')
        img_array = np.asfarray(all_values[1:]).reshape((28,28))
    else:
        img_array = scipy.misc.imread('C:/Users/Mishan/Documents/NNL/TESTPICS/testpic'+'{0:05d}'.format(number)+'.png', flatten = True)
        img_array = 255.0 - img_array
        img_array = (img_array/255.0*0.99)+0.01
    plt.imshow(img_array, cmap='Greys', interpolation='None')


In [41]:

def DOT(a,b):
    return a+b

In [42]:
a = np.array([1,2], dtype=np.float32)
b = np.array([3,-4], dtype=np.float32)
DOT(a,b)

array([ 4., -2.], dtype=float32)

In [43]:
class neuralNetwork:
    
    
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate,epochs = 1, dropout = False):
        self.inodes = inputnodes
        self.onodes = outputnodes
        self.hnodes = hiddennodes
        self.lr = learningrate
        self.wih = (np.random.rand(self.hnodes,self.inodes) - 0.5)
        self.who = (np.random.rand(self.onodes,self.hnodes) - 0.5)
        self.activation_function = lambda x: scipy.special.expit(x)
        self.scr = 1
        self.errlist = []
        self.referlist = []
        self.taste = 1
        self.epochs = epochs
        self.dropout = dropout

        pass
    
    def train(self,inputs_list,targets_list):
        
        WIH = copy.deepcopy(self.wih)
        WHO = copy.deepcopy(self.who)
        if self.dropout == True:
            for i in WIH:
                for j in i:
                    if random.randint(1,2) == 1:
                        j = 0
            for i in WHO:
                for j in i:
                    if random.randint(1,2) == 1:
                        j = 0
        
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        hidden_inputs = np.dot(WIH,inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(WHO,hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = np.dot(WHO.T, output_errors)
        
        #погнали
        self.who += self.lr*np.dot((output_errors*final_outputs*(1.0 - final_outputs)), np.transpose(hidden_outputs))
        self.wih += self.lr*np.dot((hidden_errors*hidden_outputs*(1.0 - hidden_outputs)), np.transpose(inputs))
        self.kernal = [self.wih,self.who,self.lr,self.scr,self.errlist,self.referlist, self.taste,self.epochs]
        #найс
        pass
    
    
    def query(self,inputs_list):
     
        inputs = np.array(inputs_list, ndmin=2).T
        hidden_inputs = np.dot(self.wih,inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.who,hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs
    
    def scrf(self,n):
        self.scr = n
        self.kernal = [self.wih,self.who,self.lr,self.scr,self.errlist,self.referlist, self.taste,self.epochs]
    
    def errlistf(self,n):
        self.errlist = n
        self.kernal = [self.wih,self.who,self.lr,self.scr,self.errlist,self.referlist, self.taste,self.epochs]
    
    def referlistf(self,n):
        self.referlist = n
        self.kernal = [self.wih,self.who,self.lr,self.scr,self.errlist,self.referlist, self.taste,self.epochs]
    
    def tastef(self,n):
        self.taste = n
        self.kernal = [self.wih,self.who,self.lr,self.scr,self.errlist,self.referlist, self.taste,self.epochs]
    


In [44]:
def kernal(n):
    return [n.wih,n.who,n.lr,n.scr,n.errlist,n.referlist, n.taste,n.epochs,n.hnodes]

In [45]:
def learning(machine,rotate = 1):
    data_file = open(ADRESS + "/CSV/mnist_train.csv",'r')
    data_list = data_file.readlines()
    data_file.close()
    for b in range(machine.epochs):
        for i in tqdm(data_list):
            all_values = i.split(',')
            inputs = (np.asfarray(all_values[1:])/255.0 * 0.99) + 0.01
            targets = np.zeros(output_nodes) + 0.01
            targets[int(all_values[0])]=0.99
            machine.train(inputs, targets)
    if rotate == 1:
        for i in data_list:
            all_values = i.split(',')
            inputs = (np.asfarray(all_values[1:])/255.0 * 0.99) + 0.01
            targets = np.zeros(output_nodes) + 0.01
            targets[int(all_values[0])]=0.99
            inputs_plus10 = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28),10,cval=0.01,reshape = False)
            plus10 = inputs_plus10.reshape(784)
            machine.train(plus10,targets)
            inputs_minus10 = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28),-10,cval=0.01,reshape = False)
            minus10 = inputs_minus10.reshape(784)
            machine.train(minus10,targets)
            pass
    return machine


def test(machine):
    test_data_file = open(ADRESS + "/CSV/mnist_test.csv",'r')
    test_data_list = test_data_file.readlines()
    test_data_file.close()
    scorecard = []
    errlist = []
    referlist = []
    for i in test_data_list:
        all_values = i.split(',')
        correct = int(all_values[0])
        inputs = (np.asfarray(all_values[1:])/255.0 * 0.99) + 0.01
        outputs = machine.query(inputs)
        label = np.argmax(outputs)
        if label == correct:
            scorecard.append(1)
        else:
            scorecard.append(0)
            errlist.append(label)
            referlist.append(correct)
            pass
        pass
    scr = sum(np.asarray(scorecard))/10000
    return scr, errlist, referlist

def keyfunc(machine,rotate = 1):
    machine = learning(machine,rotate = 1)
    a,b,c = test(machine)
    machine.scrf(a) 
    machine.errlistf(b)
    machine.referlistf(c) 
    
def test1(machine):
    a,b,c = test(machine)
    machine.scrf(a) 
    machine.errlistf(b)
    machine.referlistf(c) 
    
def taste(machine,name):
    pathresults = ADRESS + "/RESULTS/"+"results-"+str(name)+str(type(machine))[17:-2]+'-e'+str(machine.epochs)+'-hn-'+str(machine.hnodes)+'-lr-'+str(machine.lr)+".txt"
    a = glob.glob(ADRESS + "/TESTPICS/testpic*.png")
    results = []
    for i in a: 
        img_array = scipy.misc.imread(i, flatten = True)
        img_data = 255.0- img_array.reshape(784)
        img_data = (img_data/255.0*0.99)+0.01
        argument =  np.argmax(machine.query(img_data))
        results.append(int(np.argmax(machine.query(img_data))))
    np.savetxt(pathresults, results,fmt="%i")
    Truth  = np.loadtxt(ADRESS + "/CSV/truth.txt")
    Results = np.loadtxt(pathresults)
    s = 0
    for i in range(70,len(Results)):
        if Results[i]==Truth[i]:
            s+=1
    machine.tastef(s/(len(Truth)-70))
    
def error_function(machine,name):
    #ERROR FUNCTION FOR STATISTICS
    DATE = datetime.datetime.today().strftime("%Y-%m-%d-%H-%M") #для названий файлов
    patherror = ADRESS + "/ERRORS/errors-"+str(name)+str(type(machine))[17:-2]+'-e'+str(machine.epochs)+'-hn-'+str(machine.hnodes)+'-lr-'+str(machine.lr)+'.txt'
    errf=[0]*10
    for i in machine.errlist:
        errf[i]+=1
    print(errf)
    np.savetxt(patherror, errf,fmt="%i")
    plt.grid(True)
    plt.hist(machine.errlist,bins = range(10))
    plt.title('Ошибочный результат')
    plt.show()
    plt.grid(True)
    plt.hist(machine.referlist,bins = range(10),color='orange')
    plt.title('Ошибочно распознала')
    plt.show()

In [46]:
def save_neural_machine(machine, name):
    #PACK NEURAL NETWORK INTO PICKLE FILE
    DATE = datetime.datetime.today().strftime("%Y-%m-%d-%H-%M") #для названий файлов
    pcklpath = ADRESS + 'PICKLES/'+str(name)+str(type(machine))[17:-2]+'-e'+str(machine.epochs)+'-hn-'+str(machine.hnodes)+'-lr-'+str(machine.lr)+'.pkl'
    obj = kernal(machine)
    output = open(pcklpath, 'wb')
    pickle.dump(obj, output, 2)
    output.close()
    
def open_neural_machine(pcklpath):
    #OPEN
    input1 = open(pcklpath, 'rb')
    DATA = pickle.load(input1)
    input1.close()
    m = neuralNetwork(input_nodes, hidden_nodes, output_nodes, DATA[2])
    m.wih = DATA[0]
    m.who = DATA[1]
    m.lr = DATA[2]
    m.scr = DATA[3]
    m.errlist = DATA[4]
    m.referlist = DATA[5]
    m.taste = DATA[6]
    m.epochs = DATA[7]
    #m.hnodes = DATA[8]
    return m 

In [47]:
def inverse(machine, number, inverse_func = scipy.special.logit):
    wihI = np.array(np.matrix(machine.wih).I)
    whoI = np.array(np.matrix(machine.who).I)
    R = np.array([0.01]*10)
    R[number]+=0.98
    Ma = np.dot(whoI,R)
    Mamax = max(Ma)
    Mamin = min(Ma)
    Length = Mamax - Mamin
    K_ = 0.98/Length
    B_ = 0.99 - K_*Mamax
    Ma = K_*Ma+B_
    D = np.dot(wihI,inverse_func(Ma))
    img_array = np.asfarray(D.reshape((28,28)))
    plt.imshow(img_array, cmap='Greys', interpolation='None')

In [48]:
input_nodes = 784
hidden_nodes = 30
output_nodes = 10
learning_rate = 0.07
rate_s = []
rate_d = []

for i in range(1,15):
        test_network_d = neuralNetwork(784,10,10,0.07,i,dropout = True)
        print(type(test_network_d))
        keyfunc(test_network_d)
        rate_d.append(test_network_d.scr)
        taste(test_network_d,'DROP')
        save_neural_machine(test_network_d,'DROP')

<class '__main__.neuralNetwork'>


100%|████████████████████████████████████| 60000/60000 [18:59<00:00, 52.65it/s]


KeyboardInterrupt: 